In [ ]:
import treelite
import tl2cgen
import sklearn.datasets as datasets
import xgboost as xgb
import numpy as np
import subprocess
import random
import subprocess
import matplotlib.pyplot as plt
import platform

In [ ]:
def create_c_array_string(array):
    array_string = "{\n"
    array_string += ",\n".join("{" + ", ".join(str(x) for x in row) + "}" for row in array)
    array_string += "\n}"
    return array_string

def count_decimals(number):
    str_num = str(number)
    if '.' not in str_num:
        return 0
    return len(str_num) - str_num.index('.') - 1

def beg_c_file_str() :
    a = """
#include <stdio.h>
#include <inttypes.h>
#include "sdkconfig.h"
#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "esp_chip_info.h"
#include "esp_flash.h"
#include "esp_system.h"
#include "esp_timer.h"
#include <stdlib.h>
#include <string.h>
#include <float.h>
#include <math.h>
#include <stdint.h>

union Entry {
  int missing;
  float fvalue;
  int qvalue;
};

double elapsed_time;
int64_t start, end;


"""
    return a

def makeTimingString(val_nr, nr_results, reuses, treedepth, rootnodes):
    a = """
//////////////////////////////////////////////

start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictN"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}
end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for N"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);

start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictQ"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}
end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for Q"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);


start = esp_timer_get_time();
for (int z = 0; z < """+str(reuses)+"""; z++) {
    for (int j = 0; j < sizeof(validation_instances"""+str(val_nr)+""")/sizeof(validation_instances"""+str(val_nr)+"""[0]); j++) {
        float result"""+str(val_nr)+"""["""+str(nr_results)+"""]={0.0f};
        predictF"""+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"""(validation_instances"""+str(val_nr)+"""[j], 0, &result"""+str(val_nr)+""");
    }
}

end = esp_timer_get_time();
elapsed_time = ((double) (end - start)) / 1000000.0;
printf("Time elapsed: %f, for F"""+str(val_nr)+'m'+str(treedepth)+'td'+str(rootnodes)+'rn'+""" \\n", elapsed_time);


//////////////////////////////////////////////
"""
    return a

def makeFunctionString(val_nr, treedepth, rootnodes, features):

    # Copy predict function from NO PARAM model to function string
    a="void predictN"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"N"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    content = lines[start_line+1:end_line]
    a = a + "".join(content) + "}\n\n"

    # Copy predict function from FLINT model to function string
    a+="void predictF"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"F"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    content = lines[start_line+1:end_line]
    a = a + "".join(content) + "}\n\n"

    # Copy predict function from QUANTIZE model to function string, with the quantize function declared first
    a = a + "".join(content) + "}\n\nint quantize"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(float val, unsigned fid) {\n"
    with open("./Trees/model"+str(val_nr)+"Q"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/quantize.c", "r") as f:
        lines = f.readlines()
    start_line = next(i for i, line in enumerate(lines) if 'int quantize(float val, unsigned fid) {' in line)
    a = a + "".join(lines[2:start_line-6])+"\n"
    content = lines[start_line+1:]
    a = a + "".join(content) + "\n"
  
    
    a+="void predictQ"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(union Entry* data, int pred_margin, float* result) {\n"
    with open("./Trees/model"+str(val_nr)+"Q"+str(treedepth)+"td"+str(rootnodes)+"rn"+"/main.c", "r") as f:
        lines = f.readlines()

    start_line = next(i for i, line in enumerate(lines) if 'void predict(union Entry* data, int pred_margin, float* result) {' in line)
    end_line = next(i for i, line in enumerate(lines) if 'if (!pred_margin) { postprocess(result);' in line)
    lines[start_line+2] = "const unsigned char is_categorical[] = {"+''.join(['0, ' for i in range(features)]) + '};\n'
    lines[start_line+5] = "      data[i].qvalue = quantize"+str(val_nr)+"m"+str(treedepth)+"td"+str(rootnodes)+"rn"+"(data[i].fvalue, i);"
    content = lines[start_line+1:end_line]




    return a

nr_instances_validation = 10
toy_datasets = [datasets.load_iris]
tree_depths = [3]
nr_root_nodes = 5

header_file_str = r'const char *dll_paths[] = {'
c_file_str = beg_c_file_str()
timing_str = r''
functions_str = r''
nr_features = []
validation_instances = []
validation_instances_str = r''
nr_validation_reuses = 300

for i, dataset in enumerate(toy_datasets):
    for depth in tree_depths:

        # Train models
        X, y = dataset(return_X_y=True)
        dtrain = xgb.DMatrix(X, label=y)
        params = {"max_depth": depth, "eta": 0.1, "objective": "multi:softprob", "eval_metric": "mlogloss", "num_class": 3}
        bst = xgb.train(params, dtrain, num_boost_round=nr_root_nodes, evals=[(dtrain, 'train')])
        model = treelite.Model.from_xgboost(bst)

        path_no_param = "./Trees/model"+str(i)+"N"+str(depth)+"td"+str(nr_root_nodes)+"rn"
        path_flint = "./Trees/model"+str(i)+"F"+str(depth)+"td"+str(nr_root_nodes)+"rn"
        path_quantize = "./Trees/model"+str(i)+"Q"+str(depth)+"td"+str(nr_root_nodes)+"rn"

        # Generate model C code
        tl2cgen.generate_c_code(model, dirpath=path_no_param, params={})
        tl2cgen.generate_c_code(model, dirpath=path_flint, params={"flint" : 1})
        tl2cgen.generate_c_code(model, dirpath=path_quantize, params={"quantize": 1})

        # Generate timing C code per predict function
        timing_str += makeTimingString(i, len(np.unique(y)), nr_validation_reuses, depth, nr_root_nodes)
        functions_str += makeFunctionString(i, depth, nr_root_nodes, X.shape[1])



    # Generate random validation instances with decimal precision equal to the maximum decimal precision of the training data
    min_values_feature = []
    max_values_feature = []
    nr_decimals = np.zeros((X.shape[0], X.shape[1]))
    max_decimals = []

    nr_features.append(X.shape[1])
    for k in range(X.shape[0]):
        for j in range(X.shape[1]):
            nr_decimals[i][j] = count_decimals(X[k][j])

    for k in range(X.shape[1]):
        max_decimals.append(max(nr_decimals[:, k]))

    for k in range(X.shape[1]):
        min_values_feature.append(X[:, k].min())
        max_values_feature.append(X[:, k].max())

    validation_instances = np.zeros((nr_instances_validation, X.shape[1]))
    for j in range(nr_instances_validation):
        for k in range(X.shape[1]):
            validation_instances[j][k] = round(random.uniform(min_values_feature[k], max_values_feature[k]), int(max_decimals[k]))
    
    validation_instances_str += 'float validation_instances'+str(i)+'[][' + str(X.shape[1]) + '] = ' + create_c_array_string(validation_instances) + ';\n'


c_file_str += functions_str
c_file_str += "void app_main(void)\n{\n"
c_file_str += validation_instances_str
c_file_str += timing_str
c_file_str += "\n}"
with open('generated_speedTestS3.c', 'w') as f:
    f.write(c_file_str)



In [ ]:
# Figure scaling so that 6 bar groups are 10 wide
plt.figure(figsize=((len(times)/3)*(10/6), 5))

x = np.arange(0, len(times), 3)

# Width of a bar 
width = 2

# Create the bar chart for each group with different colors
colors = ['red', 'green', 'blue']
labels = ['No parameters', 'Flint', 'Quantize']
for i in range(3):
    plt.bar(x + i*(width/3), times[i::3], color=colors[i], width=width/3, label=labels[i])

# Assuming 'names' is your array of group names
dataset_names = [func.__name__.replace('datasets.', '') for func in toy_datasets]
dataset_names = [f"{name}{depth}" for name in dataset_names for depth in tree_depths]

for i in range(len(dataset_names)):
    dataset_names[i] = dataset_names[i].replace('load_', '')

architecture = platform.architecture()

# Set the x-tick labels to the group names and remove the x-axis
plt.xticks(x-0.5 + width/2, dataset_names)
plt.tick_params(axis='x', length=0)

plt.ylabel('Seconds')
plt.title('Inference times')
plt.text(0, -0.1, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
labels = ['Quantization', 'Inference']
cols=int(len(qtimes)/3)
fig, axs = plt.subplots(1, cols, figsize=(cols*2.2, 4))



# Assuming 'names' is your array of group names
dataset_names = [func.__name__.replace('datasets.', '') for func in toy_datasets]
dataset_names = [f"{name}{depth}" for name in dataset_names for depth in tree_depths]

for i in range(len(dataset_names)):
    dataset_names[i] = dataset_names[i].replace('load_', '')

architecture = platform.architecture()



for i in range(cols):
    cft = round(qtimes[3*i]/times[3*i+2]*100)
    axs[i].pie([qtimes[3*i+1],qtimes[3*i+2]], autopct='%1.1f%%', startangle=90)
    axs[i].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    axs[i].text(0.5, 0.1, dataset_names[i], size=12, ha="center", transform=axs[i].transAxes)
    axs[i].text(0.5, 0.05, "CFT = "+str(cft)+"%", size=10, ha="center", transform=axs[i].transAxes)
# Add a legend
plt.legend(labels, loc="best")
fig.suptitle('Quantization time breakdown', weight='bold', size='large', y=0.98, x=0.3)
fig.text(0.145, 0.92, 'Validation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nNumber of root nodes: '+str(nr_root_nodes)+'\nSystem info: '+str(architecture)+'\nCTF = Compared to Full Time', size=9, ha='left', va='top')



plt.show()

In [ ]:
dataset_names = [func.__name__.replace('datasets.', '') for func in toy_datasets]
for i in range(len(dataset_names)):
    dataset_names[i] = dataset_names[i].replace('load_', '')


times_no_parameters = [np.mean(times[i*3::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_flint = [np.mean(times[i*3+1::len(tree_depths)*3]) for i in range(len(tree_depths))]
times_quantize = [np.mean(times[i*3+2::len(tree_depths)*3]) for i in range(len(tree_depths))]

architecture = platform.architecture()
# Create the line plot for each group with different colors
plt.plot(range(len(tree_depths)), times_no_parameters, color='red', marker='*', label='No parameters')
plt.plot(range(len(tree_depths)), times_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(tree_depths)), times_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(tree_depths)), tree_depths)

plt.xlabel('Maximum tree depth')
plt.ylabel('Time [s]')
plt.title('Average inference time per tree depth')

plt.text(0, -0.15, 'Datasets used: '+str(dataset_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()

In [ ]:
times_per_nr_features= []

indices = sorted(range(len(nr_features)), key=nr_features.__getitem__)
sorted_nr_features = sorted(nr_features)

dataset_names = [func.__name__.replace('datasets.', '') for func in toy_datasets]
for i in range(len(dataset_names)):
    dataset_names[i] = dataset_names[i].replace('load_', '')

times_no_parameters = [[np.mean(times[i*len(tree_depths)*3:i*len(tree_depths)*3+len(tree_depths)*3:3]) for i in range(len(tree_depths))][j] for j in indices]
times_flint = [[np.mean(times[i*len(tree_depths)*3+1:i*len(tree_depths)*3+len(tree_depths)*3+1:3]) for i in range(len(tree_depths))][j] for j in indices]
times_quantize = [[np.mean(times[i*len(tree_depths)*3+2:i*len(tree_depths)*3+len(tree_depths)*3+2:3]) for i in range(len(tree_depths))][j] for j in indices]



# Create the line plot for each group with different colors
plt.plot(range(len(sorted_nr_features)), times_no_parameters, color='red', marker='*', label='No parameters')
plt.plot(range(len(sorted_nr_features)), times_flint, color='green', marker='^', label='Flint')
plt.plot(range(len(sorted_nr_features)), times_quantize, color='blue', marker='s', label='Quantize')

# Set the x-tick labels to the values in tree_depths
plt.xticks(range(len(sorted_nr_features)), sorted_nr_features)

plt.xlabel('Number of features')
plt.ylabel('Time [s]')
plt.title('Average inference time per number of features')

plt.text(0, -0.15, 'Datasets used: '+str(dataset_names)+'.\nValidation instances per set: '+str(nr_instances_validation)+'\nValidation set reuses: '+str(nr_validation_reuses)+'\nSystem info: '+str(architecture), ha='left', va='top', transform=plt.gca().transAxes)

# Display the legend
plt.legend()

plt.show()